<a href="https://colab.research.google.com/github/WilhelmWillie/i-love-produce/blob/master/i_love_produce_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing our good old TensorFlow libraries

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers

import tensorflow_hub as hub

import os 

In [0]:
# Download training data

TRAINING_DATA_ZIP = "https://github.com/WilhelmWillie/i-love-produce/raw/master/training_data.zip"
zip_dir = tf.keras.utils.get_file('training_data.zip', origin=TRAINING_DATA_ZIP, extract=True)

In [6]:
# Set up paths for training and validation directory
base_dir = os.path.join(os.path.dirname(zip_dir), 'training_data')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

print(base_dir, train_dir, validation_dir)

/root/.keras/datasets/training_data /root/.keras/datasets/training_data/train /root/.keras/datasets/training_data/validation


In [0]:
# Set up batches 
IMAGE_RES = 299
BATCH_SIZE = 5

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1) 

validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)